In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [34]:
# 1️⃣ Load Dataset
# Read only the needed columns and handle missing values encoded as -200
cols = ['CO(GT)', 'C6H6(GT)', 'NOx(GT)', 'NO2(GT)', 'T']
data = pd.read_csv("../content/AirQualityUCI.csv", sep=';', decimal=',', usecols=lambda c: c in cols)
# Replace sensor missing-value code -200 with NaN, then drop rows missing in our columns
data = data.replace(-200, np.nan).dropna(subset=cols)
print(f"Loaded rows after cleaning: {len(data)}")

Loaded rows after cleaning: 6941


In [35]:
data.head(10)

,CO(GT),C6H6(GT),NOx(GT),NO2(GT),T
0,2.6,11.9,166.0,113.0,13.6
1,2.0,9.4,103.0,92.0,13.3
2,2.2,9.0,131.0,114.0,11.9
3,2.2,9.2,172.0,122.0,11.0
4,1.6,6.5,131.0,116.0,11.2
5,1.2,4.7,89.0,96.0,11.2
6,1.2,3.6,62.0,77.0,11.3
7,1.0,3.3,62.0,76.0,10.7
8,0.9,2.3,45.0,60.0,10.7
11,0.7,1.1,16.0,28.0,11.0


In [36]:
# 2️⃣ Select Features and Target
features = ['CO(GT)', 'C6H6(GT)', 'NOx(GT)', 'NO2(GT)']
X = data[features].values
y = data['T'].values  # Temperature as example target
print(X.shape, y.shape)

(6941, 4) (6941,)


In [37]:
# 3️⃣ Split Dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
# 4️⃣ Scale Features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [39]:
# 5️⃣ Build Model
model = Sequential([
    Dense(16, activation='relu', input_shape=(4,)),
    Dense(8, activation='relu'),
    Dense(1)  # predicting continuous value
])

/home/hridoy/venv/lib/python3.13/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [40]:
# 6️⃣ Compile
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [41]:
# 7️⃣ Train
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

print(f"Stopped at epoch: {len(history.history['loss'])}")

Epoch 1/100


139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 351.8200 - mae: 16.5147 - val_loss: 293.9779 - val_mae: 14.6207
Epoch 2/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 192.8785 - mae: 11.1615 - val_loss: 123.7036 - val_mae: 8.9089
Epoch 3/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 104.2722 - mae: 8.0847 - val_loss: 82.0335 - val_mae: 7.2169
Epoch 4/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 69.2185 - mae: 6.5908 - val_loss: 56.2070 - val_mae: 5.9606
Epoch 5/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 51.9140 - mae: 5.7218 - val_loss: 45.8919 - val_mae: 5.3939
Epoch 6/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 44.8541 - mae: 5.2933 - val_loss: 41.3940 - val_mae: 5.1241
Epoch 7/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 41.3781 - mae: 5.0527 - val_loss: 38.9494 - val_mae: 4.9550
Epoch 8/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 39.1717 - mae: 4.9197 - val_loss: 37.3297 - val_mae: 4.8603
Epoch 9/100
139/139 ━━━━━━━━━━━━━━━━

In [42]:
# 8️⃣ Evaluate
loss, mae = model.evaluate(X_test, y_test)
print(f"✅ Test MAE: {mae:.2f}")

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 981us/step - loss: 27.7907 - mae: 4.1646
✅ Test MAE: 4.16


In [43]:
# 9️⃣ Predict
sample = np.array([[2.5, 15, 90, 45]])  # Example gas levels
sample_scaled = scaler.transform(sample)
pred = model.predict(sample_scaled)
print(f"🌡️ Predicted Temperature: {pred[0][0]:.2f} °C")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
🌡️ Predicted Temperature: 25.46 °C
